In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#import matplotlib.pyplot as plt
import requests
from geopy.geocoders import Nominatim
import time

In [2]:
glob.glob('./*/*/*.*')

['.\\Tables_brutes\\Communes_adjacentes\\communes_adjacentes_2022.csv',
 '.\\Tables_brutes\\Gare_de_voyageurs\\referentiel-gares-voyageurs.csv']

In [93]:
# Utilisation de l'API geo.API.gouv.fr
# Retourne les infos sur la commune en fonction de son nom
def info_communes(nom):
    endpoint = "https://geo.api.gouv.fr/communes?nom="
    # Create the web API’s URL
    authenticated_endpoint = endpoint+nom
    # Get the web API’s reply to the endpoint
    try: 
        api_response = requests.get(authenticated_endpoint).json()
    except:
        api_response = [{'nom': None, 'code': None, 'codeDepartement': None, 'siren': None, 'codeEpci': None, 'codeRegion': None, 'codesPostaux': [None], 'population': 0, '_score': 0}]
    if not api_response : 
        api_response = [{'nom': None, 'code': None, 'codeDepartement': None, 'siren': None, 'codeEpci': None, 'codeRegion': None, 'codesPostaux': [None], 'population': 0, '_score': 0}]
    # print(api_response[0]['code'])
    return api_response

def info_communes_code_postal(code_postal):
    endpoint = "https://geo.api.gouv.fr/communes?codePostal="
    # Create the web API’s URL
    authenticated_endpoint = endpoint+code_postal
    # Get the web API’s reply to the endpoint
    try: 
        api_response = requests.get(authenticated_endpoint).json()
    except:
        api_response = [{'nom': None, 'code': None, 'codeDepartement': None, 'siren': None, 'codeEpci': None, 'codeRegion': None, 'codesPostaux': [None], 'population': 0, '_score': 0}]
    if not api_response : 
        api_response = [{'nom': None, 'code': None, 'codeDepartement': None, 'siren': None, 'codeEpci': None, 'codeRegion': None, 'codesPostaux': [None], 'population': 0, '_score': 0}]
    # print(api_response[0]['code']) # COde postal
    return api_response

# Retourne les infos sur l'adresse et la commune en fonction des coordonnées lat , lon
def get_rev_commune(lat,lon):
    endpoint = "https://api-adresse.data.gouv.fr/reverse/?"
    # Create the web API’s URL
    authenticated_endpoint = endpoint+'lon='+str(lon)+'&lat='+str(lat)
    # Get the web API’s reply to the endpoint
    api_response = requests.get(authenticated_endpoint).json()
    # print(api_response)
    return api_response

In [94]:
# Utilisation de l'API open Street Map Nominatim
def get_location_info(address):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(address, addressdetails=True)

    if location:
        latitude = location.latitude
        longitude = location.longitude
        raw_address = location.raw.get('address', {})
        country = raw_address.get('country', None)
        return latitude, longitude, raw_address, country
    else:
        return None, None,None,None

def get_rev_location_info(lat,lon):
     geolocator = Nominatim(user_agent="my_geocoder")
     location = geolocator.reverse(f"{lat}, {lon}")
     if location:
        latitude = location.latitude
        longitude = location.longitude
        raw_address = location.raw.get('address', {})
        country = raw_address.get('country', None)
        return latitude, longitude, raw_address, country
     else:
        return 0.0,0.0,'not found','not found'

In [95]:
# function that will compare a string of words of geographical locations to a list of strings and return the best match using transfomer
def compare_strings(input_string, list_of_strings):
    input_string = input_string.lower().replace('-',' ')
    # split the input string into words
    input_words = input_string.split()
    # initialize the best match score to 0
    best_match_score = 0
    # initialize the best match string to an empty string
    best_match_string = ""
    best_index = 0
    # loop through each string in the list of strings
    for i,string in enumerate(list_of_strings):
        string = string.lower().replace('-',' ')
        # split the string into words
        string_words = string.split()
        # initialize the match score to 0
        match_score = 0
        # loop through each word in the input string
        for input_word in input_words:
            # loop through each word in the string
            for string_word in string_words:
                # if the input word is a substring of the string word, increment the match score
                if input_word in string_word:
                    match_score += 1
        # if the match score is higher than the best match score, update the best match score and best match string
        if match_score > best_match_score:
            best_match_score = match_score
            best_match_string = string
            best_index = i
    # return index of best_match_sting in list and best_match_string
    return int(best_index), best_match_string

def Trouve_code_insee(CP,Nom_com):
    try:
        CP = int(CP)
        # Si le 0 devant le code postal a été arrondi, on le rajoute
        if len(str(CP))<5:
            code_postal = '0'+ str(CP)
        else:
            code_postal = CP
        # On recharche la liste des communes avec ce code_postal
        list_com = info_communes_code_postal(str(code_postal))
        # Si la liste des communes trouvées n'est pas vide
        if list_com[0]['nom'] is not None:
            list_noms = [com['nom'] for com in info_communes_code_postal(str(code_postal))]
            # On cherche parmi le nom des communes tourvées le best match avec le nom recherché
            i, best_match = compare_strings(Nom_com, list_noms)
            # on prend le code insee du best match des communes
            code_insee = list_com[i]['code']
        else:
            # Si pas de commune trouvées avec ce code postal on met None
            best_match, code_insee = None, None
        # df_carb.loc[j, '2_code_commune_n'] = code_insee
        print(code_postal,Nom_com,"---- best match -->", best_match, code_insee)
    except:
        print('Error', CP, Nom_com)
        code_insee = None
    #
    return code_insee

# Analyse des datas

## Bornes de recharge électrique en France

https://public.opendatasoft.com/explore/dataset/mobilityref-france-irve-220/information/?disjunctive.nom_amenageur&disjunctive.nom_operateur&disjunctive.nom_enseigne&disjunctive.implantation_station&disjunctive.prise_type_ef&disjunctive.prise_type_2&disjunctive.prise_type_combo_ccs&disjunctive.prise_type_chademo&disjunctive.prise_type_autre&disjunctive.gratuit&disjunctive.paiement_acte&disjunctive.paiement_cb&disjunctive.paiement_autre&disjunctive.condition_acces&disjunctive.reservation&disjunctive.raccordement&disjunctive.com_arm_name&disjunctive.nom_epci&disjunctive.dep_name&disjunctive.reg_name&basemap=jawg.light&location=2,0.08175,8.27828&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJsaW5lIiwiZnVuYyI6IkNPVU5UIiwieUF4aXMiOiJuYnJlX3BkYyIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6IiNGRjUxNUEifV0sInhBeGlzIjoiZGF0ZV9taXNlX2VuX3NlcnZpY2UiLCJtYXhwb2ludHMiOm51bGwsInRpbWVzY2FsZSI6Im1vbnRoIiwic29ydCI6IiIsImNvbmZpZyI6eyJkYXRhc2V0IjoibW9iaWxpdHlyZWYtZnJhbmNlLWlydmUtMjIwIiwib3B0aW9ucyI6eyJkaXNqdW5jdGl2ZS5ub21fYW1lbmFnZXVyIjp0cnVlLCJkaXNqdW5jdGl2ZS5ub21fb3BlcmF0ZXVyIjp0cnVlLCJkaXNqdW5jdGl2ZS5ub21fZW5zZWlnbmUiOnRydWUsImRpc2p1bmN0aXZlLmltcGxhbnRhdGlvbl9zdGF0aW9uIjp0cnVlLCJkaXNqdW5jdGl2ZS5wcmlzZV90eXBlX2VmIjp0cnVlLCJkaXNqdW5jdGl2ZS5wcmlzZV90eXBlXzIiOnRydWUsImRpc2p1bmN0aXZlLnByaXNlX3R5cGVfY29tYm9fY2NzIjp0cnVlLCJkaXNqdW5jdGl2ZS5wcmlzZV90eXBlX2NoYWRlbW8iOnRydWUsImRpc2p1bmN0aXZlLnByaXNlX3R5cGVfYXV0cmUiOnRydWUsImRpc2p1bmN0aXZlLmdyYXR1aXQiOnRydWUsImRpc2p1bmN0aXZlLnBhaWVtZW50X2FjdGUiOnRydWUsImRpc2p1bmN0aXZlLnBhaWVtZW50X2NiIjp0cnVlLCJkaXNqdW5jdGl2ZS5wYWllbWVudF9hdXRyZSI6dHJ1ZSwiZGlzanVuY3RpdmUuY29uZGl0aW9uX2FjY2VzIjp0cnVlLCJkaXNqdW5jdGl2ZS5yZXNlcnZhdGlvbiI6dHJ1ZSwiZGlzanVuY3RpdmUucmFjY29yZGVtZW50Ijp0cnVlLCJkaXNqdW5jdGl2ZS5jb21fYXJtX25hbWUiOnRydWUsImRpc2p1bmN0aXZlLm5vbV9lcGNpIjp0cnVlLCJkaXNqdW5jdGl2ZS5kZXBfbmFtZSI6dHJ1ZSwiZGlzanVuY3RpdmUucmVnX25hbWUiOnRydWV9fX1dLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlLCJ0aW1lc2NhbGUiOiIifQ%3D%3D

In [ ]:
df_elec = pd.read_csv('.\\Tables_brutes\\Fichier consolidé des Bornes de Recharge pour Véhicules Électriques (IRVE) – schéma 2.2.0\\mobilityref-france-irve-220.csv',sep=';')

In [ ]:
df_elec.shape

In [ ]:
df_elec[8:12]

In [ ]:
df_elec.columns

In [ ]:
df_elec.dtypes

In [ ]:
df_elec['ID PDC itinérance'].unique()

In [ ]:
df_elec['ID PDC local'].unique()

In [ ]:
selected_col = [ 'Nom opérateur', 'Nom enseigne', 
        
        'ID station itinérance', 'ID station local','Nom station', 'Implantation station', 
        'Date mise en service','Nombre Points de charge',
               
                
       'Gratuit','Paiement CB','Paiement autre','Tarification', 'Condition accès',
       'Réservation', 'Horaires', 'Accessibilité PMR', 'Restriction gabarit',
       'Station deux roues',

         'ID PDC itinérance', 'ID PDC local',
        'Puissance nominale', 'Prise type E/F', 'Prise type 2', 'Prise type combo ccs', 'Prise type chademo',
         'Prise type autre', 

        'Adresse station',  
        'consolidated_longitude', 'consolidated_latitude',      
        "Code de la commune ou de l'arrondissement","Nom de la commune ou de l'arrondissement municipal",
        'Code EPCI',"Nom de l'EPCI",  
        'Code Officiel Département', 'Nom Officiel Département', 
        'Code Officiel Région','Nom Officiel Région',
        'Code INSEE commune',
        
        'Coordonnées', 
        'Paiement acte', 'Raccordement', 'Num PDL', 
        'Observations', 'Date de Mise à Jour', 'cable_t2_attache','Dernière modification', 
        'Nom aménageur',
        'consolidated_code_postal', 'consolidated_commune','consolidated_is_lon_lat_correct',
        'coord', 'longitude', 'latitude','point_geo', 
                  
               ]

In [ ]:
df_elec[df_elec["Code de la commune ou de l'arrondissement"]=='13201']

### Analyse du fichier et des valeurs manquantes

In [ ]:
# Valeurs uniques par colonne
df_elec_analyse = []
manquantes = df_elec.isna().sum()
for t in df_elec.columns:
    row = {}
    row['Variable']=t
    row['type']= df_elec[t].dtypes
    row['Nb_val_unique']=len(df_elec[t].unique())
    row['Nb_manquant']=manquantes[t]
    df_elec_analyse.append(row)
df_elec_analyse = pd.DataFrame(df_elec_analyse)
df_elec_analyse.sort_values(by='Nb_manquant',ascending=False)

In [ ]:
# Valeurs manquantes
plt.figure(figsize=(22,9))
plt.bar(x= df_elec_analyse.sort_values(by='Nb_manquant',ascending=False)['Variable'], height = df_elec_analyse.sort_values(by='Nb_manquant',ascending=False)['Nb_manquant'])
plt.xticks(rotation=90)
plt.grid()
plt.ylabel('Valeurs Manquantes (-)')

In [ ]:
# Station ou le code de la commune est manquante
pd.options.display.max_colwidth = 1000
print(len(df_elec[df_elec["Code de la commune ou de l'arrondissement"].isna()]))
df_elec.loc[df_elec["Code de la commune ou de l'arrondissement"].isna(),['Adresse station','consolidated_longitude', 'consolidated_latitude',
        'Code INSEE commune',"Nom de la commune ou de l'arrondissement municipal",'consolidated_code_postal', 'consolidated_commune'  
               ]]

In [ ]:
df_elec_analyse.sort_values(by='Nb_val_unique',ascending=False)

In [ ]:
# Valeurs uniques
plt.figure(figsize=(22,9))
plt.bar(x= df_elec_analyse.sort_values(by='Nb_val_unique',ascending=False)['Variable'], height =df_elec_analyse.sort_values(by='Nb_val_unique',ascending=False)['Nb_val_unique'])
plt.xticks(rotation=90)
plt.grid()
plt.ylabel('Nb Valeurs Uniques (-)')

In [ ]:
for c in df_elec.columns:
    print(c)
    print(df_elec[c].unique().shape)
    # print(df_elec[c].unique())
    print('-----------------------------')

In [ ]:
for c in df_elec.columns:
    print(c)
    print(df_elec[c].unique().shape)
    print(df_elec[c].unique())
    print('-----------------------------')

### Traitement des valeurs manquantes

On ne va s'intéresser qu'à la France Métropolitaine, on commence donc par supprimer les autres régions

In [ ]:
df_elec['Nom Officiel Région'].unique()

In [ ]:
df_elec['Nom Officiel Région'].value_counts()

In [ ]:
# Nous allons traiter que la France métropolitaine
df_elec= df_elec[(df_elec['Nom Officiel Région']!='Guyane') & (df_elec['Nom Officiel Région']!='Martinique') & (df_elec['Nom Officiel Région']!='Guadeloupe') & (df_elec['Nom Officiel Région']!='La Réunion') & (df_elec['Nom Officiel Région']!='Corse')]

In [ ]:
df_elec.shape

Détermination des infos manquantes concernant la localisation

In [ ]:
loc_col =['Adresse station',  
        'consolidated_longitude', 'consolidated_latitude',      
        "Code de la commune ou de l'arrondissement","Nom de la commune ou de l'arrondissement municipal",
        'Code EPCI',"Nom de l'EPCI",  
        'Code Officiel Département', 'Nom Officiel Département', 
        'Code Officiel Région','Nom Officiel Région',
          'Code INSEE commune',
         ]

Code INSEE Commune

In [ ]:
# Il n'y a pas de valeurs manquantes dans les adresses
print(len(df_elec[df_elec['Adresse station'].isna()]))

In [ ]:
# Il manque beacoup de valeurs dans ce champ
print(len(df_elec[df_elec["Code INSEE commune"].isna()]))

In [ ]:
# IL ne manque pas beacoup de valeurs dans ce chanmps
print(len(df_elec[df_elec["Code de la commune ou de l'arrondissement"].isna()]))

In [ ]:
df_elec[df_elec["Code de la commune ou de l'arrondissement"]=='75114']

In [ ]:
df_elec["Code de la commune ou de l'arrondissement"].value_counts()

In [ ]:
# Lignes ou la cellule Code de commune est NaN
print(len(df_elec.loc[df_elec["Code de la commune ou de l'arrondissement"].isna(),loc_col]))
df_elec.loc[df_elec["Code de la commune ou de l'arrondissement"].isna(),loc_col]

In [ ]:
def check_code_commune(dfx):
    if pd.notna(dfx["Code de la commune ou de l'arrondissement"]):
        code_INSEE = dfx["Code de la commune ou de l'arrondissement"]
    elif pd.notna(dfx["Code INSEE commune"]):
        code_INSEE = dfx["Code INSEE commune"]
    elif pd.notna(dfx["consolidated_code_postal"]):
        code_INSEE = info_communes(dfx["consolidated_code_postal"])
    else:
        try:
            infos_adresses = get_location_info(dfx['Adresse station'])
            # print(infos_adresses)
            code_postal = infos_adresses[2]['city']
            code_INSEE=info_communes[0]['code']
        except:
            code_INSEE=None
    return code_INSEE

In [ ]:
# On chercher le code commune d'abord dans le tableau, puis avec l'API.gouv.fr
def check_code_commune(dfx):
    if pd.notna(dfx["Code de la commune ou de l'arrondissement"]):
        code_INSEE = dfx["Code de la commune ou de l'arrondissement"]
    elif pd.notna(dfx["Code INSEE commune"]):
        code_INSEE = dfx["Code INSEE commune"]
    elif pd.notna(dfx["consolidated_code_postal"]):
        code_INSEE = info_communes(dfx["consolidated_code_postal"])
    else:
        try:
            infos_adresses = get_location_info(dfx['Adresse station'])
            # print(infos_adresses)
            code_postal = infos_adresses[2]['city']
            code_INSEE=info_communes[0]['code']
        except:
            code_INSEE=None
    return code_INSEE
df_elec['code_commune_n']=df_elec.apply(lambda x: check_code_commune(x), axis=1)

In [ ]:
# Ensuite on recherche par lat lon 
for i, x in df_elec[(df_elec['code_commune_n'].isna()) & (df_elec['latitude'].notna()) & (df_elec['longitude'].notna())].iterrows():
    print(i)
    try: 
        df_elec.loc[i,'code_commune_n'] = get_rev_commune(df_elec.loc[i,'latitude'],df_elec.loc[i,'longitude'])['features'][0]['properties']['citycode']
    except:
        pass

In [ ]:
i= 10656
get_rev_commune(df_elec.loc[i,'latitude'],df_elec.loc[i,'longitude'])

In [ ]:
#df_elec.to_csv(r'C:\Users\nimod\OneDrive - Université de Technologie de Troyes\Fil_rouge\Tables_traitees\Stations_electriques_avec_code_commune.csv',sep=';',index=False)

In [ ]:
df_elec[df_elec['code_commune_n'].isna()]

In [ ]:
# Lignes ou la cellule Code commune checked == 0
loc_col.append('code_commune_n')
print(len(df_elec.loc[df_elec["code_commune_n"]==0,loc_col]))
df_elec.loc[df_elec["code_commune_n"]==0,loc_col]

In [ ]:
# On va supprimer les lignes où le code_commune_checked==0 car ce n'est pas en France métrolpolitaine
df_elec =df_elec[df_elec["code_commune_n"]!=0]

In [ ]:
df_elec.shape

Lat, Lon et adresse

Les valeurs les "moins" manquantes sont la latitude/ longitude ainsi que l"adresse, on va donc commencer par corriger tout cela

In [ ]:
# Il n'y a pas de valeurs manquantes dans consolidated latitiude
print(len(df_elec[df_elec['consolidated_latitude'].isna()]))

In [ ]:
# Il n'y a pas de valeurs manquantes dans consolidated longitude
print(len(df_elec[df_elec['consolidated_longitude'].isna()]))

In [ ]:
#M Par contre il y a des valeurs à 0 en latitude ou en longitude
print(len(df_elec.loc[(df_elec['consolidated_latitude']==0.0) | (df_elec['consolidated_longitude']==0.0),loc_col]))
df_elec.loc[(df_elec['consolidated_latitude']==0.0) | (df_elec['consolidated_longitude']==0.0),loc_col]

In [ ]:
def correct_lat_lon(dfx):
    if (dfx['consolidated_latitude']==0.0) | (dfx['consolidated_longitude']==0.0):
        dfx['lat_n'], dfx['lon_n'] = get_location_info(dfx['Adresse station'])[0:2]
    else:
        dfx['lat_n'], dfx['lon_n'] = dfx['consolidated_latitude'], dfx['consolidated_longitude']
    return dfx

In [ ]:
df_elec=df_elec.apply(lambda x: correct_lat_lon(x),axis=1)

In [ ]:
#M Par contre il y a des valeurs à 0 en latitude ou en longitude
loc_col.append('lat_n')
loc_col.append('lon_n')
print(len(df_elec.loc[(df_elec['lat_n']==0.0) | (df_elec['lon_n']==0.0),loc_col]))
df_elec.loc[(df_elec['lat_n']==0.0) | (df_elec['lon_n']==0.0),loc_col]

In [ ]:
get_location_info('2 RUE LOUIS BLERIOT 87220 FEYTIAT')

In [ ]:
for i,x in df_elec.loc[df_elec["Nom Officiel Région"].isna(),:].iterrows():
    print(i, x['Adresse station'], x['consolidated_longitude'], x['consolidated_latitude'])
    if (x['consolidated_longitude']!=0.0) & (x['consolidated_latitude']!=0.0):
        print(get_rev_location_info(x['consolidated_latitude']	,x['consolidated_longitude']))
    else:
        print(get_location_info(df_elec_2.loc[i,'Adresse station']))
    print('--------------------------------------')

In [ ]:
for i,x in df_elec.loc[(df_elec['consolidated_latitude']==0.0) | (df_elec['consolidated_longitude']==0.0),:].iterrows():
    address = x['Adresse station']
    print(address,get_location_info(address) )

In [ ]:
selected_col = [ 
        'consolidated_longitude', 'consolidated_latitude',

        'Adresse station',
                
        "Code de la commune ou de l'arrondissement","Nom de la commune ou de l'arrondissement municipal",

        'consolidated_code_postal', 'consolidated_commune','consolidated_is_lon_lat_correct',
        'Code INSEE commune',          
               ]

Regions

In [ ]:
print(len(df_elec_2[df_elec_2["Nom Officiel Région"].isna()]))

In [ ]:
df_elec_2.loc[df_elec_2["Nom Officiel Région"].isna(),loc_col]

In [ ]:
get_location_info(df_elec_2.loc[1515,'Adresse station'])

In [ ]:
get_rev_location_info(42.720000	,9.460000)

In [ ]:
            # print(infos_adresses)
            code_postal = infos_adresses[2]['city']
            code_INSEE=info_communes[0]['code']

Adresse Station

In [ ]:
print(len(df_elec[df_elec["Adresse station"].isna()]))

In [ ]:
df_elec["Adresse station"].value_counts()

In [ ]:
df_elec.dtypes

In [ ]:
df_elec.describe()

In [ ]:
df_elec[df_elec.duplicated(keep=False)]

In [ ]:
df_elec.duplicated(subset=['ID station itinérance'],keep=False)

In [ ]:
print(df_elec['Nom Officiel Région'].unique().shape)
df_elec['Nom Officiel Région'].unique()

In [ ]:
print(df_elec['Nom aménageur'].unique().shape)
df_elec['Nom aménageur'].unique()

In [ ]:
df_elec['Nom aménageur'].unique()

In [ ]:
df_elec['SIREN aménageur'].unique()

In [ ]:
df_elec['Puissance nominale'].unique()

In [ ]:
df_elec['Nombre Points de charge'].unique()

In [ ]:
df_elec.value_counts('Nombre Points de charge')

In [ ]:
df_elec.value_counts('Code INSEE commune')

In [ ]:
df_elec.shape

In [ ]:
manquantes['Nom aménageur']

In [ ]:
df_elec[df_elec['Nombre Points de charge']==505]

In [ ]:
df_elec['Date mise en service'].unique()

In [ ]:
df_elec['Date mise en service'].unique())

## Infos générales communes 2013

https://public.opendatasoft.com/explore/dataset/correspondance-code-insee-code-postal/table/

In [ ]:
df_commune_info = pd.read_csv('.\\Tables_brutes\\Correspondance Code INSEE - Code Postal\\correspondance-code-insee-code-postal.csv',sep=';')

In [ ]:
df_commune_info[:3]

In [ ]:
df_commune_info.dtypes

In [ ]:
df_commune_info['Département'] = df_commune_info.apply(lambda x: x['Département'].replace("['","").replace("']","").replace('["',"").replace('"]',""),axis=1)
df_commune_info['Région'] = df_commune_info.apply(lambda x: x['Région'].replace("['","").replace("']","").replace('["',"").replace('"]',""),axis=1)
df_commune_info['Statut'] = df_commune_info.apply(lambda x: x['Région'].replace("['","").replace("']","").replace('["',"").replace('"]',""),axis=1)

In [ ]:
df_commune_info.shape

In [ ]:
len(df_commune_info['Code INSEE'].unique())

In [ ]:
df_commune_info.columns

In [ ]:
# On crée le df_code pour avoir la correspondance nom de commune avec le code INSEE
df_code = df_commune_info[['Code INSEE','Commune']]

In [ ]:
# On va créer deux colonnes lat lon en splittant geo point
def split_lat_lon(s):
   try:
       lat,lon = s.split(',')
   except:
        lat,lon =np.NaN, np.NaN
   return float(lat), float(lon)
df_commune_info['lat']=df_commune_info.apply(lambda x: split_lat_lon(x['geo_point_2d'])[0],axis=1)
df_commune_info['lon']=df_commune_info.apply(lambda x: split_lat_lon(x['geo_point_2d'])[1],axis=1)

In [ ]:
df_commune_info.loc[df_commune_info['Commune']=='DEMI-QUARTIER',:]

In [ ]:
df_commune_info['Densité_hab_par_km2'] = df_commune_info['Population']*1000 /  (df_commune_info['Superficie'] /100)

In [ ]:
df_commune_info.loc[df_commune_info['Commune']=='DEYVILLERS',:]

In [ ]:
# Valeurs uniques par colonne
df_commune_info_analyse = []
manquantes = df_commune_info.isna().sum()
for t in df_commune_info.columns:
    row = {}
    row['Variable']=t
    row['type']= df_commune_info[t].dtypes
    row['Nb_val_unique']=len(df_commune_info[t].unique())
    row['Nb_manquant']=manquantes[t]
    df_commune_info_analyse.append(row)
df_commune_info_analyse = pd.DataFrame(df_commune_info_analyse)
df_commune_info_analyse
df_commune_info_analyse.sort_values(by='Nb_manquant',ascending=False)

In [ ]:
#df_commune_info.to_csv(r'C:\Users\nimod\OneDrive - Université de Technologie de Troyes\Fil_rouge\Tables_traitees\Infos_communes.csv',sep=';',index=False)

## Population  communes dernier recensement

In [ ]:
df_pop = pd.read_csv('.\\Tables_brutes\\Population_des_communes_françaises\\population-francaise-communes_dernier_recensement.csv',sep=';')

In [ ]:
df_pop.shape

In [ ]:
df_pop.dtypes

In [ ]:
df_pop[:3]

In [ ]:

Code région	Nom de la région	Code département	Code arrondissement départemental	Code canton	Code commune	Nom de la commune	Population municipale	Population comptée à part	Population totale	Année recensement	Année utilisation	Code INSEE (commune ou arrondissement)	Superficie de la commune	Statut	Code INSEE de la commune	Nom de la commune IGN	Nom du département IGN	Nom de la région.1	Code EPCI	Code EPCI.1	EPCI
0	84.0	Auvergne-Rhône-Alpes	01	2.0	8.0	1.0	L' Abergement-Clémenciat	767.0	18.0	785.0	2016	2019	01001	15.65	Commune	01001	NaN	NaN	NaN	200035210	200035210	CC Chalaronne Centre
1	84.0	Auvergne-Rhône-Alpes	01	1.0	1.0	2.0	L' Abergement-de-Varey	243.0	1.0	244.0	2016	2019	01002	9.13	Commune	01002	L'Abergement-de-Varey	AIN	AUVERGNE-RHONE-ALPES	240100883	240100883	CC de la Plaine de l'Ain
2	84.0	Auvergne-Rhône-Alpes	01	1.0	1.0	4.0	Ambérieu-en-Bugey	14081.0	437.0	14518.0	2016	2019	01004	24.49	Commune	01004	Ambérieu-en-Bugey	AIN	AUVERGNE-RHONE-ALPES	240100883	240100883	CC de la Plaine de l'Ain
df_pop[df_pop['Code INSEE (commune ou arrondissement)']!=df_pop['Code INSEE de la commune']]

In [ ]:
df_pop[df_pop['Code EPCI']!=df_pop['Code EPCI.1']].shape

In [ ]:
df_pop[df_pop['Code INSEE (commune ou arrondissement)']=='13055']

In [ ]:
# Valeurs uniques par colonne
df_pop_analyse = []
manquantes = df_pop.isna().sum()
for t in df_pop.columns:
    row = {}
    row['Variable']=t
    row['type']= df_pop[t].dtypes
    row['Nb_val_unique']=len(df_pop[t].unique())
    row['Nb_manquant']=manquantes[t]
    df_pop_analyse.append(row)
df_pop_analyse = pd.DataFrame(df_pop_analyse)
df_pop_analyse
# df_pop_analyse.sort_values(by='Nb_manquant',ascending=False)

In [ ]:
# On filtre pour ne garder que les champs que l'on veut
# On va garder que les codes des communes et des arrondissements
df_pop = df_pop[['Code région', 'Nom de la région', 'Code département',
         'Code commune',
       'Nom de la commune', 'Population municipale',
       'Population comptée à part', 'Population totale', 'Année recensement',
       'Année utilisation', 'Code INSEE (commune ou arrondissement)',
       'Superficie de la commune', 'Statut', 
       'Code EPCI', 'EPCI']]

In [ ]:
df_pop['Densité_hab_par_km2'] = df_pop['Population municipale'] /  df_pop['Superficie de la commune']

In [ ]:
df_pop.loc[df_pop['Nom de la commune']=='Deyvillers',:]

In [ ]:
df_pop.columns

In [ ]:
# df_pop.to_csv(r'C:\Users\nimod\OneDrive - Université de Technologie de Troyes\Fil_rouge\Tables_traitees\Population-Superficie_communes.csv',sep=';',index=False)

## Communes adjacentes

In [ ]:
df_commune_adj = pd.read_csv('.\\Tables_brutes\\Communes_adjacentes\\communes_adjacentes_2022.csv',sep=',')

In [ ]:
df_commune_adj.shape

In [ ]:
df_commune_adj[:3]

## Prix_vente_m2_communes

In [ ]:
df_commune_immo = pd.read_csv('.\\Tables_brutes\\Prix_moyen_m2_vente\\dvf-communes-2019.csv',sep=';')

In [ ]:
df_commune_immo.dtypes

In [ ]:
df_commune_immo.shape

In [ ]:
df_commune_immo[:5]

In [ ]:
df_commune_immo.columns

In [ ]:
df_commune_immo.dtypes

In [ ]:
df_commune_immo[df_commune_immo.INSEE_COM=='88132']

In [ ]:
df_commune_immo['nb_digits_code_INSEE'] = df_commune_immo['INSEE_COM'].str.len()
df_commune_immo['nb_digits_code_INSEE'].unique()

In [ ]:
# On va corriger le code commune dans ce fichier, car le premier 0 a été oublié
def add_0(x):
    if len(str(x))==4:
        x = "0"+str(x)
    else:
        x =str(x)
    return x 
df_commune_immo['code_commune_n'] = df_commune_immo.apply(lambda x: add_0(x['INSEE_COM']), axis=1)

In [ ]:
# Valeurs uniques par colonne
df_commune_immo_analyse = []
manquantes = df_commune_immo.isna().sum()
for t in df_commune_immo.columns:
    row = {}
    row['Variable']=t
    row['type']= df_commune_immo[t].dtypes
    row['Nb_val_unique']=len(df_commune_immo[t].unique())
    row['Nb_manquant']=manquantes[t]
    df_commune_immo_analyse.append(row)
df_commune_immo_analyse = pd.DataFrame(df_commune_immo_analyse)
df_commune_immo_analyse.sort_values(by='Nb_manquant',ascending=False)

In [ ]:
df_commune_immo['nb_digits_code_INSEE'] = df_commune_immo['code_commune_n'].str.len()
df_commune_immo['nb_digits_code_INSEE'].unique()

In [ ]:
#df_commune_immo.to_csv(r'C:\Users\nimod\OneDrive - Université de Technologie de Troyes\Fil_rouge\Tables_traitees\PrixMoy_m2_avec_code_commune.csv', sep=';',index=False)

## Stations carburants en France à J-7

https://public.opendatasoft.com/explore/dataset/prix_des_carburants_j_7/information/?sort=update

In [ ]:
df_carb = pd.read_csv('.\\Tables_brutes\\Prix des carburants - J-7\\prix_des_carburants_j_7.csv',sep=';')

In [ ]:
df_carb[:10]

In [ ]:
df_carb.columns

In [ ]:
df_carb.shape

In [ ]:
df_carb['Identifiant'].unique().shape

In [ ]:
df_carb['Identifiant'].unique()

In [ ]:
# On va créer deux colonnes lat lon en splittant geo point
def split_lat_lon(s):
   try:
       lat,lon = s.split(',')
   except:
        lat,lon =np.NaN, np.NaN
   return float(lat), float(lon)
df_carb['lat']=df_carb.apply(lambda x: split_lat_lon(x['Geo Point'])[0],axis=1)
df_carb['lon']=df_carb.apply(lambda x: split_lat_lon(x['Geo Point'])[1],axis=1)

In [ ]:
# Valeurs uniques par colonne
df_carb_analyse = []
manquantes = df_carb.isna().sum()
for t in df_carb.columns:
    row = {}
    row['Variable']=t
    row['type']= df_carb[t].dtypes
    row['Nb_val_unique']=len(df_carb[t].unique())
    row['Nb_manquant']=manquantes[t]
    df_carb_analyse.append(row)
df_carb_analyse = pd.DataFrame(df_carb_analyse)
df_carb_analyse.sort_values(by='Nb_manquant',ascending=False)

In [ ]:
## On commence par rechercher dans le df_code s'il y a des villes déjà enregistrées
# df_carb = pd.merge(df_carb, df_code, left_on='Ville', right_on='Commune', how='left')
# df_carb['code_commune_n']=df_carb['Code INSEE']

In [ ]:
# Ensuite on Recherche par ville sur l'API gouv.fr 
def code(x):
    if pd.isna(x['code_commune_n']):
        c = info_communes(x['Ville'])[0]['code']
    else:
        c =  x['code_commune_n']
    return c
df_carb['code_commune_n'] = df_carb.apply(lambda x: code(x),axis=1)

In [ ]:
# ATTENTION CODE LONG
# Ensuite on recherche par lat lon 
for i, x in df_carb[(df_carb['code_commune_n'].isna()) & (df_carb['lat'].notna()) & (df_carb['lon'].notna())].iterrows():
    print(i)
    try: 
        df_carb.loc[i,'code_commune_n'] = get_rev_commune(df_carb.loc[i,'lat'],df_carb.loc[i,'lon'])['features'][0]['properties']['citycode']
    except:
        pass

In [ ]:
# ATTENTION CODE LONG
#Pour ceux qui manque toujours on va essayer la recherche avec  l'adresse, on pourrait aussi essayer l'API google map mais c'est payant
for i, x in df_carb[(df_carb['code_commune_n'].isna())].iterrows():
    print(i,f"{x['Adresse']} {x['Code Postal']} {x['Ville']}")
    df_carb.loc[i,'lat'],df_carb.loc[i,'lon'],_,_ = get_location_info(f"{x['Adresse']} {x['Code Postal']} {x['Ville']}")
    time.sleep(1)

In [ ]:
get_location_info('56 AVE DU DR PICAUD 6150 CANNES LA BOCCA')

In [ ]:
get_location_info('33 rue du commandant René Mouchotte 75014 PARIS')

In [ ]:
get_rev_commune(48.8375511,2.3187555)

In [ ]:
info_communes('Paris')

In [ ]:
df_carb[(df_carb['code_commune_n'].isna())]

In [ ]:
#df_carb.to_csv(r'C:\Users\nimod\OneDrive - Université de Technologie de Troyes\Fil_rouge\Tables_traitees\Stations_carburant_avec_code_commune.csv', sep=';',index=False)

In [ ]:

#df_carb['code_commune_n']= df_carb.apply(lambda x: info_communes(x['Ville'])[0]['code'],axis=1)

In [ ]:
info_communes(df_carb.loc[5,'Ville'])

In [ ]:
get_rev_location_info(df_carb.loc[0,'lat'],df_carb.loc[0,'lon'])

In [ ]:
get_rev_commune(df_carb.loc[7702,'lat'],df_carb.loc[7702,'lon'])['features'][0]['properties']['citycode']

In [ ]:
get_rev_commune(df_carb.loc[7702,'lat'],df_carb.loc[7702,'lon'])

## Gare ferroviaire française

In [187]:
df_gare = pd.read_csv('./Tables_brutes/Gare_de_voyageurs/referentiel-gares-voyageurs.csv',sep=';')
rename = {col:"7_"+col for col in df_gare.columns}
df_gare.rename(columns = rename, inplace=True)

In [188]:
df_gare.shape

(3133, 24)

In [189]:
# Il y a 234 gares qui ont une fin de validité plateforme, nous allons les supprimer
df_gare[df_gare['7_Date fin validité plateforme'].notna()].shape

(234, 24)

In [190]:
df_gare[df_gare['7_Date fin validité plateforme'].notna()]

,7_Code plate-forme,7_Code gare,7_Code UIC,7_Date fin validité plateforme,7_Intitulé plateforme,7_Code postal,7_Code Commune,7_Commune,7_Code département,7_Département,...,7_RG,7_Intitulé gare,7_Intitulé fronton de gare,7_DTG,7_Région SNCF,7_Unité gare,7_UT,7_Nbre plateformes,7_TVS,7_WGS 84
20,00095-1,95,87775494,2015-12-31,Bessèges,30160.0,37.0,Bessèges,30.0,Gard,...,GARES C LANGUEDOC ROUSSILLON,Bessèges,Bessèges,Agence Grand Sud,NaN,NaN,NaN,1,NaN,"44.2884691, 4.1005901"
24,00110-1,110,87775122,2013-12-31,Aigues Vives,30670.0,4.0,Aigues-Vives,30.0,Gard,...,GARES C LANGUEDOC ROUSSILLON,Aigues Vives,Aigues Vives,NaN,NaN,NaN,NaN,1,NaN,NaN
30,00148-1,148,87764472,2014-01-01,Le Teil,7400.0,319.0,Teil,7.0,Ardèche,...,GARES C RHONE,Le Teil,Le Teil,DRG AURA-BFC,NaN,NaN,LE TEIL (ARDECHE) GARE,1,NaN,"44.5525285, 4.685992"
46,00233-1,233,87756007,2014-12-14,Nice Saint Roch,6300.0,88.0,Nice,6.0,Alpes-Maritimes,...,GARES C COTE AZUR,Nice Saint Roch,Nice Saint Roch,Agence Grand Sud,NaN,NaN,NaN,1,NaN,"43.715342, 7.286869"
74,00402-1,402,87741447,2012-12-31,La Bridoire,73520.0,NaN,NaN,NaN,NaN,...,GARES C ALPES,La Bridoire,La Bridoire,NaN,NaN,NaN,NaN,1,NaN,NaN
87,00469-1,469,87726794,2012-12-31,Beauzac,43590.0,NaN,NaN,NaN,NaN,...,GARES C AUVERGNE,Beauzac,Beauzac,NaN,NaN,NaN,NaN,1,NaN,NaN
95,00496-1,496,87725812,2012-12-31,St Jean sur Veyle,1290.0,NaN,NaN,NaN,NaN,...,GARES C ALPES,St Jean sur Veyle,St Jean sur Veyle,NaN,NaN,NaN,NaN,1,NaN,NaN
98,00520-1,520,87723551,2012-12-31,La Boisse,1120.0,NaN,NaN,NaN,NaN,...,GARES C RHONE ALPES,La Boisse,La Boisse,NaN,NaN,NaN,NaN,1,NaN,NaN
101,00531-1,531,87723361,2012-12-31,Chandieu Toussieu,69780.0,NaN,NaN,NaN,NaN,...,GARES C RHONE ALPES,Chandieu Toussieu,Chandieu Toussieu,NaN,NaN,NaN,NaN,1,NaN,NaN
122,00628-1,628,87718049,2015-12-31,Miserey-Salines,25480.0,381.0,Miserey-Salines,25.0,Doubs,...,GARES C FRANCHE COMTE,Miserey-Salines,Miserey-Salines,Agence Centre Est Rhône Alpin,NaN,NaN,NaN,1,NaN,"47.2854733, 5.9636746"


In [191]:
# Le reste des gares n'a pas de fin de validité donc on suppose qu'elle est toujours en service
df_gare[df_gare['7_Date fin validité plateforme'].isna()].shape

(2899, 24)

In [192]:
# Nous allons supprimer les gares qui ont une fin de validaté de plateforme
# How to drop rows that are NOT NA in df_gare dataframe
df_gare = df_gare[df_gare['7_Date fin validité plateforme'].isna()]

In [193]:
# IL ne nous reste plus que 2899 gares
df_gare.shape

(2899, 24)

In [194]:
df_gare[df_gare['7_Intitulé gare']!=df_gare['7_Commune']]

,7_Code plate-forme,7_Code gare,7_Code UIC,7_Date fin validité plateforme,7_Intitulé plateforme,7_Code postal,7_Code Commune,7_Commune,7_Code département,7_Département,...,7_RG,7_Intitulé gare,7_Intitulé fronton de gare,7_DTG,7_Région SNCF,7_Unité gare,7_UT,7_Nbre plateformes,7_TVS,7_WGS 84
0,00001-1,1,87988717,NaN,Lycée Henri Sellier,93190.0,46.0,Livry-Gargan,93.0,Seine-Saint-Denis,...,GARES B IDF LIGNE T4,Lycée Henri Sellier,Lycée Henri Sellier,DGIF,REGION DE PARIS-EST,NaN,LIVRY GARGAN GARE LYCEE HENRI SELLIER TR,1,LHS,"48.91646, 2.51504"
3,00020-1,20,87784744,NaN,Thuès les Bains,66360.0,36.0,Canaveilles,66.0,Pyrénées-Orientales,...,GARES C LANGUEDOC ROUSSILLON,Thuès les Bains,Thuès les Bains,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,THUES LES BAINS GARE,1,THB,"42.5288009, 2.249094"
4,00023-1,23,87784710,NaN,Joncet,66193.0,193.0,Serdinya,66.0,Pyrénées-Orientales,...,GARES C LANGUEDOC ROUSSILLON,Joncet,Joncet,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,JONCET GARE,1,JCT,"42.5615245, 2.3111464"
5,00027-1,27,87784637,NaN,Prades - Molitg-les-Bains,66500.0,149.0,Prades,66.0,Pyrénées-Orientales,...,GARES C LANGUEDOC ROUSSILLON,Prades - Molitg-les-Bains,Prades - Molitg-les-Bains,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,PRADES MOLITG LES BAINS GARE,1,PDS,"42.6170988, 2.4294184"
9,00043-1,43,87783670,NaN,Belvezet,48170.0,27.0,Mont Lozère et Goulet,48.0,Lozère,...,GARES C LANGUEDOC ROUSSILLON,Belvezet,Belvezet,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,BELVEZET GARE,1,BVZ,"44.5617971, 3.7518919"
10,00051-1,51,87783456,NaN,Tournemire - Roquefort,12250.0,282.0,Tournemire,12.0,Aveyron,...,GARES C MIDI PYRENEES,Tournemire - Roquefort,Tournemire - Roquefort,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,TOURNEMIRE ROQUEFORT GARE,1,TEO,"43.9698002, 3.0166261"
13,00069-1,69,87781559,NaN,Les Cabrils,34233.0,233.0,Roqueredonde,34.0,Hérault,...,GARES C LANGUEDOC ROUSSILLON,Les Cabrils,Les Cabrils,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,LES CABRILS GARE,1,KBI,"43.779707, 3.1860831"
14,00071-1,71,87781534,NaN,Montpaon,12540.0,155.0,Fondamente,12.0,Aveyron,...,GARES C MIDI PYRENEES,Montpaon,Montpaon,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,MONTPAON GARE,1,MNP,"43.8685236, 3.1097221"
15,00078-1,78,87781088,NaN,Leucate La Franqui,11370.0,202.0,Leucate,11.0,Aude,...,GARES C LANGUEDOC ROUSSILLON,Leucate La Franqui,Leucate la Franqui,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,LEUCATE LA FRANQUI GARE,1,LEU,"42.9327725, 3.0128555"
18,00085-1,85,87775825,NaN,Le Cailar,30740.0,59.0,Cailar,30.0,Gard,...,GARES C LANGUEDOC ROUSSILLON,Le Cailar,Le Cailar,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,LE CAILAR GARE,1,LCI,"43.6834141, 4.2354236"


In [195]:
df_gare['7_Code département'].unique()

array([93., 66., 48., 12., 34., 11., 30., 84., 26.,  5., 38.,  6., 83.,
       13., 74., 39.,  1., 73., 43., 63., 42., 71., 69., 25., 21., 89.,
       58.,  3., 45., 91., 77., 94., 64., 40., 65., 15., 81., 46., 82.,
       31.,  9., 23., 36., 24., 19., 16., 87., 47., 33., 18., 41., 86.,
       37., 75., 17., 79., 85., 49., 44., 53., 50., 22., 35., 56., 29.,
       61., 27., 14., 78., 76., 72., 28., 92., 95., 60., 59., 62., 80.,
        2., 67., 54., 55., 57., 70., 90., 68., 51.,  8., 88., 52., 10.,
        4., nan, 32.])

In [196]:
df_gare['7_Code Commune'].unique()

array([ 46.,  25., 167.,  36., 193., 149., 174.,  16.,   8.,  27., 282.,
       243., 147., 233., 155., 202.,  32., 276.,  59., 341., 255., 132.,
       198., 169., 333.,  54., 119.,  70., 179., 394., 165.,  29., 163.,
       162.,  23.,  91.,  88.,  31.,  33.,  62., 105.,   4.,  56., 102.,
        55.,   5., 242., 143., 164., 250., 224., 210.,  43., 158., 478.,
        72., 181., 121.,   1., 315., 109., 157., 195., 453., 178., 269.,
       124., 278., 308.,  13.,  11.,  99.,  95.,  97., 218., 372., 543.,
       509., 449.,  89., 378.,  39., 111.,  44., 234.,   9., 293., 411.,
       271., 429., 527., 258., 489., 336., 397., 515., 290., 607., 194.,
       464., 183., 292., 418., 663., 627., 231., 473., 286., 298.,  57.,
       264., 295., 306., 122., 187., 191.,  69., 458., 431.,  48., 237.,
       257., 206., 160., 188., 419.,  37., 445.,  67., 296.,  45., 228.,
       521., 687.,   2., 230.,  61., 400., 101., 470.,  82., 301.,  10.,
        65., 304., 305.,  15.,  83., 128., 555., 18

In [197]:
def build_code(dep,com):
    try:
        com = str(int(com))
        dep = str(int(dep))
        if len(com)==3:
            code_insee = dep+com
        elif len(com)==2:
            code_insee = dep+'0'+com
        elif len(com)==1:
            code_insee = dep+'00'+com
    except:
        print('error')
        code_insee = np.nan
    print(dep,com,code_insee)
    return code_insee

In [198]:
df_gare.columns

Index(['7_Code plate-forme', '7_Code gare', '7_Code UIC',
       '7_Date fin validité plateforme', '7_Intitulé plateforme',
       '7_Code postal', '7_Code Commune', '7_Commune', '7_Code département',
       '7_Département', '7_Longitude', '7_Latitude', '7_Segment DRG',
       '7_Niveau de service', '7_RG', '7_Intitulé gare',
       '7_Intitulé fronton de gare', '7_DTG', '7_Région SNCF', '7_Unité gare',
       '7_UT', '7_Nbre plateformes', '7_TVS', '7_WGS 84'],
      dtype='object')

In [199]:
df_gare['7_code_commune_n'] = df_gare.apply(lambda c: build_code(c['7_Code département'],c['7_Code Commune']),axis=1)

93 46 93046
66 25 66025
66 167 66167
66 36 66036
66 193 66193
66 149 66149
66 174 66174
66 16 66016
66 8 66008
48 27 48027
12 282 12282
12 243 12243
34 147 34147
34 233 34233
12 155 12155
11 202 11202
34 32 34032
30 276 30276
30 59 30059
30 341 30341
30 255 30255
30 132 30132
48 198 48198
30 169 30169
34 333 34333
84 54 84054
84 119 84119
84 16 84016
26 198 26198
5 70 5070
5 46 5046
5 179 5179
38 394 38394
26 165 26165
26 333 26333
6 29 6029
6 163 6163
6 162 6162
6 23 6023
6 91 6091
6 32 6032
6 27 6027
6 88 6088
83 31 83031
83 33 83033
83 62 83062
13 105 13105
13 4 13004
13 56 13056
13 102 13102
13 55 13055
13 5 13005
38 242 38242
74 56 74056
74 56 74056
74 56 74056
74 143 74143
74 164 74164
74 250 74250
74 224 74224
74 210 74210
74 43 74043
74 158 74158
74 243 74243
39 478 39478
1 72 1072
73 181 73181
73 121 73121
38 1 38001
38 315 38315
73 109 73109
73 8 73008
43 157 43157
63 195 63195
63 453 63453
63 178 63178
63 269 63269
63 124 63124
63 278 63278
63 308 63308
43 13 43013
42 11 420

In [200]:
df_gare.head()

,7_Code plate-forme,7_Code gare,7_Code UIC,7_Date fin validité plateforme,7_Intitulé plateforme,7_Code postal,7_Code Commune,7_Commune,7_Code département,7_Département,...,7_Intitulé gare,7_Intitulé fronton de gare,7_DTG,7_Région SNCF,7_Unité gare,7_UT,7_Nbre plateformes,7_TVS,7_WGS 84,7_code_commune_n
0,00001-1,1,87988717,NaN,Lycée Henri Sellier,93190.0,46.0,Livry-Gargan,93.0,Seine-Saint-Denis,...,Lycée Henri Sellier,Lycée Henri Sellier,DGIF,REGION DE PARIS-EST,NaN,LIVRY GARGAN GARE LYCEE HENRI SELLIER TR,1,LHS,"48.91646, 2.51504",93046
1,00007-1,7,87784876,NaN,Bourg-Madame,66760.0,25.0,Bourg-Madame,66.0,Pyrénées-Orientales,...,Bourg-Madame,Bourg-Madame,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,BOURG MADAME GARE,1,BMD,"42.4324069, 1.9486704",66025
2,00011-1,11,87784835,NaN,Saillagouse,66800.0,167.0,Saillagouse,66.0,Pyrénées-Orientales,...,Saillagouse,Saillagouse,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,SAILLAGOUSE GARE,1,SGW,"42.4568855, 2.0321126",66167
3,00020-1,20,87784744,NaN,Thuès les Bains,66360.0,36.0,Canaveilles,66.0,Pyrénées-Orientales,...,Thuès les Bains,Thuès les Bains,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,THUES LES BAINS GARE,1,THB,"42.5288009, 2.249094",66036
4,00023-1,23,87784710,NaN,Joncet,66193.0,193.0,Serdinya,66.0,Pyrénées-Orientales,...,Joncet,Joncet,DRG Occitanie Sud,REGION LANGUEDOC-ROUSSILLON,UG Est Occitanie,JONCET GARE,1,JCT,"42.5615245, 2.3111464",66193


In [201]:
# Nous avons 121 communes pour lequelles le code insee manque
df_gare[df_gare['7_code_commune_n'].isna()].shape

(9, 25)

In [202]:
df_gare[df_gare['7_code_commune_n'].isna()]

,7_Code plate-forme,7_Code gare,7_Code UIC,7_Date fin validité plateforme,7_Intitulé plateforme,7_Code postal,7_Code Commune,7_Commune,7_Code département,7_Département,...,7_Intitulé gare,7_Intitulé fronton de gare,7_DTG,7_Région SNCF,7_Unité gare,7_UT,7_Nbre plateformes,7_TVS,7_WGS 84,7_code_commune_n
707,00323-1,323,87747204,NaN,Virieu sur Bourbre,38730.0,NaN,NaN,NaN,NaN,...,Virieu sur Bourbre,Virieu sur Bourbre,DRG AURA-BFC,REGION ALPES,UG Alpes,VIRIEU SUR BOURBRE GARE,1,VRB,"45.491703, 5.4721424",NaN
1019,01911-1,1911,87396093,NaN,La Hutte Coulombiers,72130.0,NaN,NaN,NaN,NaN,...,La Hutte Coulombiers,La Hutte Coulombiers,DRG B-CVL-PDL,REGION PAYS DE LA LOIRE,UG Pays de la Loire,LA HUTTE COULOMBIERS GARE,1,LHU,"48.303495, 0.1056788",NaN
1093,02296-1,2296,87313338,NaN,Hargicourt - Pierrepont,80500.0,NaN,NaN,NaN,NaN,...,Hargicourt - Pierrepont,Hargicourt - Pierrepont,DRG HDF-Normandie,REGION PICARDIE,UG Picardie,HARGICOURT PIERREPONT GARE,1,HGI,"49.7138593, 2.5317673",NaN
1886,03220-1,3220,87743872,NaN,Le Mans hôpital-université,72000.0,NaN,NaN,NaN,NaN,...,Le Mans hôpital-université,NaN,DRG B-CVL-PDL,NaN,NaN,NaN,1,NaN,NaN,NaN
1978,00407-1,407,87741256,NaN,Aiguebelle,73220.0,NaN,NaN,NaN,NaN,...,Aiguebelle,Aiguebelle,DRG AURA-BFC,REGION ALPES,UG Alpes,AIGUEBELLE GARE,1,AEB,"45.5437225, 6.3067175",NaN
2183,01540-1,1540,87486431,NaN,Olonne-sur-Mer,85340.0,NaN,NaN,NaN,NaN,...,Olonne-sur-Mer,Olonne-sur-Mer,DRG B-CVL-PDL,REGION PAYS DE LA LOIRE,UG Pays de la Loire,OLONNE SUR MER GARE,1,OSM,"46.5486605, -1.76738",NaN
2504,03229-1,3229,87583617,NaN,La Couronne - Charente,16400.0,NaN,NaN,NaN,NaN,...,La Couronne - Charente,NaN,DRG Nlle Aquitaine,NaN,NaN,NaN,1,NaN,NaN,NaN
2505,03290-1,3290,87725473,NaN,Boulazac,24750.0,NaN,NaN,NaN,NaN,...,Boulazac,Boulazac,DRG Nlle Aquitaine,NaN,UG Aquitaine,BOULAZAC HALTE,1,NaN,NaN,NaN
2701,01038-1,1038,87611491,NaN,Porta,66146.0,NaN,NaN,NaN,NaN,...,Porta,NaN,DRG Occitanie Sud,NaN,NaN,PORTA GARE,1,NaN,NaN,NaN


In [203]:
# Méthode on recherche par le code postal, parmi la liste des noms de communes avec le bon code postal
# On prend celle qui a le meilleure match de mot
for j, c in df_gare[df_gare['7_code_commune_n'].isna()].iterrows():
    df_gare.loc[j, '7_code_commune_n'] = Trouve_code_insee(c['7_Code postal'],c['7_Intitulé plateforme'])

38730 Virieu sur Bourbre ---- best match --> val de virieu 38560
72130 La Hutte Coulombiers ---- best match -->  72011
80500 Hargicourt - Pierrepont ---- best match -->  80032
72000 Le Mans hôpital-université ---- best match --> le mans 72181
73220 Aiguebelle ---- best match -->  73007
85340 Olonne-sur-Mer ---- best match --> l'île d'olonne 85112
16400 La Couronne - Charente ---- best match --> la couronne 16113
24750 Boulazac ---- best match --> boulazac isle manoire 24053
66146 Porta ---- best match --> None None


In [204]:
# Nous avons désormais 81 communes pour lequelles le code insee manque, qui correspondent au manque de code postal sauf 1
print(df_gare[df_gare['7_code_commune_n'].isna()].shape)
print(df_gare[df_gare['7_Code postal'].isna()].shape)

(1, 25)
(0, 25)


In [205]:
df_gare[df_gare['7_code_commune_n'].isna()]

,7_Code plate-forme,7_Code gare,7_Code UIC,7_Date fin validité plateforme,7_Intitulé plateforme,7_Code postal,7_Code Commune,7_Commune,7_Code département,7_Département,...,7_Intitulé gare,7_Intitulé fronton de gare,7_DTG,7_Région SNCF,7_Unité gare,7_UT,7_Nbre plateformes,7_TVS,7_WGS 84,7_code_commune_n
2701,01038-1,1038,87611491,NaN,Porta,66146.0,NaN,NaN,NaN,NaN,...,Porta,NaN,DRG Occitanie Sud,NaN,NaN,PORTA GARE,1,NaN,NaN,None


In [206]:
for j, c in df_gare[df_gare['7_code_commune_n'].isna()].iterrows():
    print(c['7_Intitulé gare'],info_communes(c['7_Intitulé gare'])[0]['code'])
    df_gare.loc[j, '7_code_commune_n'] = info_communes(c['7_Intitulé gare'])[0]['code']

Porta 66146


In [207]:
# Nous avons désormais 47 communes pour lequelles le code insee manque
print(df_gare[df_gare['7_code_commune_n'].isna()].shape)

(0, 25)


In [208]:
df_gare[df_gare['7_code_commune_n'].isna()]

,7_Code plate-forme,7_Code gare,7_Code UIC,7_Date fin validité plateforme,7_Intitulé plateforme,7_Code postal,7_Code Commune,7_Commune,7_Code département,7_Département,...,7_Intitulé gare,7_Intitulé fronton de gare,7_DTG,7_Région SNCF,7_Unité gare,7_UT,7_Nbre plateformes,7_TVS,7_WGS 84,7_code_commune_n


In [209]:
# On va tenter de trvouer la commune par l'intitulé de la 7_Intitulé gare
for j, c in df_gare[df_gare['7_code_commune_n'].isna()].iterrows():
    try:
        latlon = get_location_info(c['7_Intitulé gare'])
        code_insee = get_rev_commune(latlon[0],latlon[1])['features'][0]['properties']['citycode']  
    except:
        code_insee = None
    print(c['7_Intitulé gare'], code_insee)
    df_gare.loc[j, '7_code_commune_n'] =code_insee

In [210]:
# Nous avons désormais 13 communes pour lequelles le code insee manque
print(df_gare[df_gare['7_code_commune_n'].isna()].shape)

(0, 25)


In [211]:
df_gare[df_gare['7_code_commune_n'].isna()]

,7_Code plate-forme,7_Code gare,7_Code UIC,7_Date fin validité plateforme,7_Intitulé plateforme,7_Code postal,7_Code Commune,7_Commune,7_Code département,7_Département,...,7_Intitulé gare,7_Intitulé fronton de gare,7_DTG,7_Région SNCF,7_Unité gare,7_UT,7_Nbre plateformes,7_TVS,7_WGS 84,7_code_commune_n


In [212]:
# On va remplir à la main
for j, c in df_gare[df_gare['7_code_commune_n'].isna()].iterrows():
    print(j,f"Gare de {c['7_Intitulé gare']}")

In [213]:
# df_gare.loc[615, '7_code_commune_n']=get_rev_commune(46.752833356877595, 6.304625492571351)['features'][0]['properties']['citycode']
# df_gare.loc[620, '7_code_commune_n']=get_rev_commune(49.28703100612136, 0.1922559220721422)['features'][0]['properties']['citycode']
# df_gare.loc[621, '7_code_commune_n']=get_rev_commune(44.85104973704001, 0.4123279869732798)['features'][0]['properties']['citycode']
# df_gare.loc[625, '7_code_commune_n']=get_rev_commune(47.406222510579575, 5.128207656666878)['features'][0]['properties']['citycode']
# #df_gare.loc[1251, '7_code_commune_n']=get_rev_commune(47.05010175704488, 6.726075978295844)['features'][0]['properties']['citycode']
# df_gare.loc[1254, '7_code_commune_n']=get_rev_commune(47.619225536092976, 6.150276009537717)['features'][0]['properties']['citycode']
# df_gare.loc[1870, '7_code_commune_n']=get_rev_commune(44.85590495532807, 0.5817771204830784)['features'][0]['properties']['citycode']
# #df_gare.loc[1878, '7_code_commune_n']=get_rev_commune(17.89944109862467, -62.85077991103902)['features'][0]['properties']['citycode']
# df_gare.loc[1882, '7_code_commune_n']=get_rev_commune(48.83421718550606, -1.2254604062154364)['features'][0]['properties']['citycode']
# df_gare.loc[2492, '7_code_commune_n']=get_rev_commune(44.98073346263664, 2.0297312623956056)['features'][0]['properties']['citycode']
# df_gare.loc[3117, '7_code_commune_n']=get_rev_commune(46.22428808559069, 2.984312008153886)['features'][0]['properties']['citycode']
# #df_gare.loc[3121, '7_code_commune_n']=get_rev_commune(58.95658553725248, 5.741945921805124)['features'][0]['properties']['citycode']
# df_gare.loc[3127, '7_code_commune_n']=get_rev_commune(46.79054264090162, -0.7325849652450389)['features'][0]['properties']['citycode']

In [214]:
# On va remplir à la main
for j, c in df_gare[df_gare['7_code_commune_n'].isna()].iterrows():
    print(j,f"Gare de {c['7_Intitulé gare']}")

In [215]:
# On supprime les dernieres car elles ne sont pas en France métropolitaine
df_gare.dropna(subset='7_code_commune_n', inplace=True)

In [216]:
# Il n'y a plus de code insee manquant
print(df_gare[df_gare['7_code_commune_n'].isna()].shape)

(0, 25)


In [217]:
# show number of na for each column of df_gare
df_gare.isna().sum()

7_Code plate-forme                   0
7_Code gare                          0
7_Code UIC                           0
7_Date fin validité plateforme    2899
7_Intitulé plateforme                0
7_Code postal                        0
7_Code Commune                       9
7_Commune                            9
7_Code département                   9
7_Département                        9
7_Longitude                         24
7_Latitude                          24
7_Segment DRG                       10
7_Niveau de service                446
7_RG                                27
7_Intitulé gare                      0
7_Intitulé fronton de gare          15
7_DTG                                1
7_Région SNCF                       42
7_Unité gare                       402
7_UT                                19
7_Nbre plateformes                   0
7_TVS                               37
7_WGS 84                            24
7_code_commune_n                     0
dtype: int64

In [218]:
df_gare.shape

(2899, 25)

In [219]:
# df_gare.to_csv('./Tables_traitees\\Gare_ferroviaire_avec_code_commune.csv', sep=';', index=False)

## Aeroport français

In [264]:
df_aero = pd.read_csv('./Tables_brutes/Aeroport_francais/liste_aeroports.csv',sep='|')
rename = {col:"8_"+col for col in df_aero.columns}
df_aero.rename(columns = rename, inplace=True)

In [265]:
df_aero.shape

(90, 10)

In [266]:
df_aero.dtypes

8_Nom aéroport           object
8_Code OACI              object
8_Code IATA              object
8_Latitude               object
8_Longitude              object
8_N° du département       int64
8_Libellé département    object
8_Région                 object
8_Ville desservie        object
8_Adresse                object
dtype: object

In [267]:
df_aero.isna().sum()

8_Nom aéroport           0
8_Code OACI              0
8_Code IATA              0
8_Latitude               0
8_Longitude              0
8_N° du département      0
8_Libellé département    0
8_Région                 0
8_Ville desservie        0
8_Adresse                0
dtype: int64

In [273]:
df_aero.head()

,8_Nom aéroport,8_Code OACI,8_Code IATA,8_Latitude,8_Longitude,8_N° du département,8_Libellé département,8_Région,8_Ville desservie,8_Adresse,8_lat_dec,8_lon_dec
0,AGEN-LA-GARENNE,LFBA,AGF,44° 10' 12,0° 35' 23,47,Lot-et-Garonne,Aquitaine,Agen,LE PASSAGE 47520,44.170000,0.589722
1,AJACCIO-NAPOLEON-BONAPARTE,LFKJ,AJA,41° 55' 12,8° 48' 0,20,Corse,Corse,Ajaccio,AJACCIO 20090,41.920000,8.800000
2,ALBI-LE-SEQUESTRE,LFCI,LBI,43° 54' 35,2° 7' 12,81,Tarn,Midi-Pyrénées,Albi,ALBI 81000,43.909722,2.120000
3,ANGERS-MARCE,LFJR,ANE,47° 33' 36,-0° 18' 36,49,Maine etLoire,Pays de la Loire,Angers,MARCE 49140,47.560000,0.310000
4,ANGOULEME-BRIE-CHAMPNIERS,LFBU,ANG,45° 43' 47,0° 13' 12,16,Charente,Poitou-Charentes,Angouleme,CHAMPNIERS 16430,45.729722,0.220000


In [285]:
# Create a function that Converts latitude and longitude from deg, min, sec into decimal
def convert_lat(lat):
    lat_deg = lat.split('°')
    lat_min = lat_deg[1].split("'")[0]
    lat_sec = lat_deg[1].split("'")[1]
    if '-' in lat_deg[0]:
        lat_dec = -(abs(float(lat_deg[0]))+float(lat_min)/60+float(lat_sec)/3600)
    else:
        lat_dec = (abs(float(lat_deg[0]))+float(lat_min)/60+float(lat_sec)/3600)
    print(lat, lat_dec)
    return lat_dec
def convert_lon(lon):
    lon_deg = lon.split('°')
    lon_min = lon_deg[1].split("'")[0]
    lon_sec = lon_deg[1].split("'")[1]
    
    if '-' in lon_deg[0]:
        lon_dec = -(abs(float(lon_deg[0]))+float(lon_min)/60+float(lon_sec)/3600)
    else:
        lon_dec = (abs(float(lon_deg[0]))+float(lon_min)/60+float(lon_sec)/3600)
    print(lon, lon_dec)
    return lon_dec

In [286]:
df_aero['8_lat_dec'] = df_aero.apply(lambda a: convert_lat(a["8_Latitude"]),axis=1)

44° 10' 12 44.169999999999995
41° 55' 12 41.919999999999995
43° 54' 35 43.90972222222222
47° 33' 36 47.559999999999995
45° 43' 47 45.72972222222222
45° 55' 47 45.92972222222222
44° 53' 59 44.89972222222222
47° 47' 54 47.79833333333333
43° 54' 35 43.90972222222222
42° 32' 59 42.54972222222222
49° 27' 0 49.45
44° 49' 12 44.82
43° 19' 12 43.32
43° 28' 11 43.469722222222224
44° 49' 47 44.82972222222222
47° 3' 36 47.059999999999995
48° 27' 0 48.45
45° 02' 28 45.04111111111111
49° 10' 12 49.169999999999995
44° 26' 53 44.448055555555555
50° 56' 52 50.94777777777777
42° 31' 12 42.519999999999996
43° 32' 59 43.54972222222222
43° 13' 11 43.219722222222224
43° 32' 59 43.54972222222222
48° 46' 24 48.77333333333333
45° 38' 24 45.64
46° 51' 35 46.859722222222224
49° 38' 59 49.64972222222222
47° 4' 47 47.07972222222222
45° 47' 23 45.78972222222222
48° 6' 35 48.109722222222224
49° 21' 35 49.359722222222224
47° 16' 12 47.269999999999996
48° 35' 24 48.59
47° 2' 23 47.03972222222222
48° 19' 47 48.3297222

In [287]:
df_aero['8_lon_dec'] = df_aero.apply(lambda a: convert_lon(a["8_Longitude"]),axis=1)

0° 35' 23 0.5897222222222223
8° 48' 0 8.8
2° 7' 12 2.12
-0° 18' 36 -0.31
0° 13' 12 0.22
6° 6' 36 6.109999999999999
2° 25' 11 2.419722222222222
3° 34' 1 3.5669444444444443
4° 54' 0 4.9
9° 28' 48 9.48
2° 6' 35 2.1097222222222225
0° 31' 12 0.52
3° 21' 0 3.35
-1° 31' 48 -1.53
-0° 43' 11 -0.7197222222222223
2° 22' 47 2.379722222222222
-4° 25' 11 -4.419722222222222
1° 28' 59 1.4830555555555556
-0° 27' 0 -0.45
1° 26' 28 1.4411111111111112
1° 51' 23 1.856388888888889
8° 47' 23 8.789722222222222
6° 57' 35 6.959722222222222
2° 18' 36 2.3099999999999996
2° 17' 24 2.29
4° 12' 22 4.206111111111111
5° 52' 47 5.879722222222223
1° 43' 11 1.7197222222222224
-1° 28' 47 -1.4797222222222224
-0° 53' 23 -0.8897222222222222
3° 9' 36 3.1599999999999997
7° 21' 36 7.359999999999999
0° 9' 36 0.16
5° 5' 23 5.089722222222222
-2° 4' 48 -2.08
5° 25' 47 5.429722222222223
6° 4' 12 6.069999999999999
9° 5' 59 9.099722222222223
5° 19' 48 5.33
-2° 23' 24 -2.39
-1° 12' 0 -1.2
-3° 28' 12 -3.47
-0° 44' 23 -0.7397222222222222

In [306]:
# On va tenter de trvouer la commune par la latitude et la longitude
for j, a in df_aero.iterrows():
    try:
        answer =  get_rev_location_info(a['8_lat_dec'],a['8_lon_dec'])
        adresse = answer[2]['road']+' '+ answer[2]['postcode']+' '+answer[2]['municipality']
        print(a['8_Nom aéroport'], adresse)
        code_insee = (Trouve_code_insee(answer[2]['postcode'],answer[2]['municipality']))
    except:
        code_insee = None
    df_aero.loc[j, '8_code_commune_n'] = code_insee

AGEN-LA-GARENNE Chemin des Primeurs 47310 Agen
47310 Agen ---- best match -->  47016
AJACCIO-NAPOLEON-BONAPARTE T 40 20129 Ajaccio
20129 Ajaccio ---- best match -->  2A032
ALBI-LE-SEQUESTRE Rue des Lilas 81990 Albi
81990 Albi ---- best match -->  81052
ANGERS-MARCE L’Océane 49140 Angers
49140 Angers ---- best match -->  49076
ANGOULEME-BRIE-CHAMPNIERS Rue Jean Mermoz 16430 Angoulême
16430 Angoulême ---- best match -->  16026
AURILLAC N 122 15000 Aurillac
15000 Aurillac ---- best match --> aurillac 15014
AUXERRE-BRANCHES Impasse du Grand Caire 89000 Auxerre
89000 Auxerre ---- best match --> auxerre 89024
AVIGNON-CAUMONT Route de Marseille 84000 Avignon
84000 Avignon ---- best match --> avignon 84007
BASTIA-PORETTA Corsu di l'Aeroportu 20290 Bastia
20290 Bastia ---- best match -->  2B042
BEAUVAIS-TILLE Avenue de l'Europe 60000 Beauvais
60000 Beauvais ---- best match --> beauvais 60057
BERGERAC-ROUMANIERE C 205 24100 Bergerac
24100 Bergerac ---- best match --> bergerac 24037
BEZIERS-VIAS 

In [307]:
df_aero[df_aero['8_code_commune_n'].isna()].shape

(8, 13)

In [308]:
df_aero[df_aero['8_code_commune_n'].isna()]

,8_Nom aéroport,8_Code OACI,8_Code IATA,8_Latitude,8_Longitude,8_N° du département,8_Libellé département,8_Région,8_Ville desservie,8_Adresse,8_lat_dec,8_lon_dec,8_code_commune_n
5,ANNECY-MEYTHET,LFLP,NCY,45° 55' 47,6° 6' 36,74,Haute-Savoie,Rhône-Alpes,Annecy,METZ T ESSY 74370,45.929722,6.110000,None
18,CAEN-CARPIQUET,LFRK,CFR,49° 10' 12,-0° 27' 0,14,Calvados,Basse-Normandie,Caen,CARPIQUET 14650,49.170000,-0.450000,None
48,LIMOGES-BELLEGARDE,LFBL,LIG,45° 51' 35,1° 10' 47,87,Haute-Vienne,Limousin,Limoges,LIMOGES 87100,45.859722,1.179722,None
59,NANTES-ATLANTIQUE,LFRS,NTE,47° 9' 35,-1° 36' 36,44,Loire-Atlantique,Pays de la Loire,Nantes,CCI NANTES BOUGUENAIS4 4340,47.159722,-1.610000,None
60,NICE-COTE-D'AZUR,LFMN,NCE,43° 40' 12,7° 12' 35,6,Alpes-Maritimes,Provence-Alpes-Côte d'Azur,Nice,CCI NCA - B.P. 3331 NICE CEDEX 36206,43.670000,7.209722,None
63,PARIS-CHARLES-DE-GAULLE,LFPG,CDG,49° 0' 35,2° 33' 0,93,Seine-Saint-Denis,Ile-de-France,Paris,BP 20102 ROISSY 95711 CEDEX,49.009722,2.550000,None
68,PERIGUEUX-BASSILLAC,LFBX,PGX,45° 12' 0,0° 49' 11,24,Dordogne,Aquitaine,Perigueux,PERIGUEUX 24000,45.200000,0.819722,None
73,RENNES-ST-JACQUES,LFRN,RNS,48° 4' 12,-1° 43' 47,35,Ille-et-Vilaine,Bretagne,Rennes,Avenue Joseph Le Brix - B.P. 29155 RENNES 35091,48.070000,-1.729722,None


In [309]:
# On va remplir à la main
for j, c in df_aero[df_aero['8_code_commune_n'].isna()].iterrows():
    print(j,f"Aeroport de {c['8_Nom aéroport']}")

5 Aeroport de ANNECY-MEYTHET
18 Aeroport de CAEN-CARPIQUET
48 Aeroport de LIMOGES-BELLEGARDE
59 Aeroport de NANTES-ATLANTIQUE
60 Aeroport de NICE-COTE-D'AZUR
63 Aeroport de PARIS-CHARLES-DE-GAULLE
68 Aeroport de PERIGUEUX-BASSILLAC
73 Aeroport de RENNES-ST-JACQUES


In [311]:
for j, c in df_aero[df_aero['8_code_commune_n'].isna()].iterrows():
    print(f"df_aero.loc[{j}, '8_code_commune_n']=get_rev_commune(XXXXXXXXXXX)['features'][0]['properties']['citycode'])")

df_aero.loc[5, '8_code_commune_n']=get_rev_commune(XXXXXXXXXXX)['features'][0]['properties']['citycode'])
df_aero.loc[18, '8_code_commune_n']=get_rev_commune(XXXXXXXXXXX)['features'][0]['properties']['citycode'])
df_aero.loc[48, '8_code_commune_n']=get_rev_commune(XXXXXXXXXXX)['features'][0]['properties']['citycode'])
df_aero.loc[59, '8_code_commune_n']=get_rev_commune(XXXXXXXXXXX)['features'][0]['properties']['citycode'])
df_aero.loc[60, '8_code_commune_n']=get_rev_commune(XXXXXXXXXXX)['features'][0]['properties']['citycode'])
df_aero.loc[63, '8_code_commune_n']=get_rev_commune(XXXXXXXXXXX)['features'][0]['properties']['citycode'])
df_aero.loc[68, '8_code_commune_n']=get_rev_commune(XXXXXXXXXXX)['features'][0]['properties']['citycode'])
df_aero.loc[73, '8_code_commune_n']=get_rev_commune(XXXXXXXXXXX)['features'][0]['properties']['citycode'])


In [319]:
df_aero.loc[5, '8_code_commune_n']=get_rev_commune(45.9312984832095, 6.1068207516703445)['features'][0]['properties']['citycode']
df_aero.loc[18, '8_code_commune_n']=14137
df_aero.loc[48, '8_code_commune_n']=get_rev_commune(45.8632471972159, 1.1818036931329354)['features'][0]['properties']['citycode']
df_aero.loc[59, '8_code_commune_n']=44020
df_aero.loc[60, '8_code_commune_n']=6088
df_aero.loc[63, '8_code_commune_n']=get_rev_commune(49.0079863208414, 2.5507500485370773)['features'][0]['properties']['citycode']
df_aero.loc[68, '8_code_commune_n']=get_rev_commune(45.19653307444863, 0.8125796355335243)['features'][0]['properties']['citycode']
df_aero.loc[73, '8_code_commune_n']=get_rev_commune(48.068407762180335, -1.7259889066705685)['features'][0]['properties']['citycode']

In [320]:
# Il n'y a plus de manquant
df_aero[df_aero['8_code_commune_n'].isna()]

,8_Nom aéroport,8_Code OACI,8_Code IATA,8_Latitude,8_Longitude,8_N° du département,8_Libellé département,8_Région,8_Ville desservie,8_Adresse,8_lat_dec,8_lon_dec,8_code_commune_n


In [321]:
# show number of na for each column of df_gare
df_aero.isna().sum()

8_Nom aéroport           0
8_Code OACI              0
8_Code IATA              0
8_Latitude               0
8_Longitude              0
8_N° du département      0
8_Libellé département    0
8_Région                 0
8_Ville desservie        0
8_Adresse                0
8_lat_dec                0
8_lon_dec                0
8_code_commune_n         0
dtype: int64

In [323]:
# df_aero.to_csv('./Tables_traitees\\Aeroports_avec_code_commune.csv', sep=';', index=False)

# XXXXXX

## Europe - Railway Station

https://public.opendatasoft.com/explore/dataset/europe-railway-station/information/

In [ ]:
df_rail = pd.read_csv(glob.glob('.\\Europe Railway Station\\*.csv')[0],sep=';')

In [ ]:
df_rail[:5]

In [ ]:
df_rail['ICC'].unique()

In [ ]:
df_rail.columns

In [ ]:
df_rail.shape

## Base Nationale des Lieux de Covoiturage en France

https://public.opendatasoft.com/explore/dataset/base-nationale-des-lieux-de-covoiturage-france/information/

In [ ]:
df_cov = pd.read_csv(glob.glob('.\\Base Nationale des Lieux de Covoiturage en France\\*.csv')[0],sep=';')

In [ ]:
df_cov[:5]

In [ ]:
df_cov.columns

In [ ]:
df_cov.shape

## Base nationale des lieux de stationnement en France

https://public.opendatasoft.com/explore/dataset/mobilityref-france-base-nationale-des-lieux-de-stationnement/table/

In [ ]:
df_stat = pd.read_csv(glob.glob('.\\Base nationale des lieux de stationnement en France\\*.csv')[0],sep=';')

In [ ]:
df_stat[:5]

In [ ]:
df_stat.columns

In [ ]:
df_stat.shape